# Metasurfaces Data Base Genetation Ansys HFSS Interface

## _Versión 0.1_
### Author: Jorge Cárdenas
### Pontificia Universidad de Valparaíso
This development is intended to provide an interface to access HFSS, in order to automate simulation and optimization processes.

## Features

- Automate the re-creation of models in aedt files.
- Create intermediate files to run simulations.
- Run simulations and gather specific metrics from the simualtor.


In [1]:

!pip install druida --upgrade
%pip install pandas


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.1 MB 10.2 MB/s eta 0:00:04
    --------------------------------------- 0.8/38.1 MB 10.6 MB/s eta 0:00:04
   - -------------------------------------- 1.4/38.1 MB 10.9 MB/s eta 0:00:04
   -- ------------------------------------- 1.9/38.1 MB 10.2 MB/s eta 0:00:04
   -- ------------------------------------- 2.5/38.1 MB 11.3 MB/s eta 0:00:04
   --- ------------------------------------ 3.0/38.1 MB 10.8 MB/s eta 0:00:04
   --- ------------------------------------ 3.5/38.1 MB 10.5 MB/s eta 0:00:04
   ---- ----------------------------------- 4.0/38.1 MB 10.7 MB/s eta 0:00:04
   ---- ----------------------------------- 4.6/38.1 MB 10.8 MB/s eta 0:00:04
   ----- ---------------------------------- 5.1/38.1 MB 11.3 MB/s eta 0:00:03
   ----- ---------------------------------- 5.7/38.1 MB 11.3 MB/s eta 0:

In [1]:
"""When working in Dev mode"""
import sys  
sys.path.insert(0, './src/')


"""when working in Production mode"""
#%pip install druidaHFSS

'when working in Production mode'

In [2]:



import os
import uuid 
from datetime import datetime


from __future__ import print_function
#from Utilities.SaveAnimation import Video
from druidaHFSS import Manager as MG



## Setup
<p>In this section we prepare all variables to set the origin and destination folders for our resulting files</p>

In [3]:
ansysPath="C:\\Program Files\\AnsysEM\\AnsysEM21.2\\Win64\\ansysedt.exe"
modelName="cross_01_freq_reflect"
modelPath=r"C:/Users/jorge/Documents/Projects Jorge C/DRUIDA PROJECT/POC/dbGeneration_v0/Models/testing-multioutput/"
exportPath= "C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\Exports"
dBPath= "C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\DBfiles\\"
project_name="meta-atom_cross_01_datageneration"
designName="HFSSDesign1"

#set simulation variables
#This is run to create iterative simulations



In [4]:
#Model builder

#ansysPath,modelName, modelPath,exportPath

Builder=MG.Builder(ansysPath=ansysPath,modelName=modelName,projectName=project_name, designName=designName,modelPath=modelPath, exportPath=exportPath)


TypeError: Builder.__init__() missing 2 required positional arguments: 'scriptPath' and 'imagesPath'

In [35]:
#ReCreate a aedt model from script 
#We are supposed to have previously worked our model out and keep a python script 
#to be used as source to create the geometry when required.

Builder.create()

# 1. A full batch of simulation for box-shaped cells

![alternatvie text](./resources/Picture1.png)

In [7]:
#Generate batch of vectors to simulate
import random

def parameters_Generator(constraints, batch_size, vector_size):
    x = [[0 for i in range(vector_size)] for j in range(batch_size)]

    for vector in x:
        for i, (k, v) in enumerate(constraints.items()):
            vector[i]=random.uniform(constraints[k]["max"], constraints[k]["min"])
            
    return x
    

    

In [9]:

batch=0
iteration=0
simfile_path= "C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\"
simfileName= "intermediateFile.py"

#set reports
reports={
    "ReflectanceTE":"(mag(S(FloquetPort1:1,FloquetPort1:1)))^2",
    "ReflectanceTM":"(mag(S(FloquetPort1:2,FloquetPort1:2)))^2"
    }

constraints = {"xize":{"max":4, "min":0.3,"nominaSl":1},
            "ySize":{"max":4, "min":0.3,"nominal":1},
            "zSize":{"max":0.08, "min":0.6,"nominal":0.5},
            "zSustrato":{"max":1.5, "min":0.5,"nominal":1}}

parameters = parameters_Generator(constraints, batch_size=3, vector_size=4)     



for parameter in parameters:
    
    
    #parameters, batch, iteration, filePath, **kwargs
    #Kwargs
    #AnsoftPath=kwargs['ansoft_path']
    #modelPath=kwargs['model_path'] 
    #outputPath=kwargs['output_path']
    #projectName=kwargs['project_name']
    #simulationID=kwargs['simulation_id']
    #variableName=kwargs['variable_name']
    #value=kwargs['value']
    #units=kwargs['units']
    #design=kwargs['design_name']

    #Create the intermediate simulation fiel
    
    simulation_id=str(uuid.uuid1())

    #set variables to modify
    variable_name='parameters'
    value=str(parameter)

    units="mm"
    
    kwargs={
        "reports":reports,
        "simulation_id":simulation_id,
       "variable_name":variable_name,
        "value" : value,
        "units" : units,
       }
    
    Builder.sim_file('', batch, iteration, simfile_path, **kwargs)
    
    Builder.simulate(simfile_path+simfileName)
    
    iteration+=iteration


"C:/Users/jorge/Documents/Projects Jorge C/DRUIDA PROJECT/POC/dbGeneration_v0/Models/testing-multioutput/meta-atom_box_01_datageneration.aedt"
The new directory is created!
"C:/Users/jorge/Documents/Projects Jorge C/DRUIDA PROJECT/POC/dbGeneration_v0/Models/testing-multioutput/meta-atom_box_01_datageneration.aedt"
The new directory is created!
"C:/Users/jorge/Documents/Projects Jorge C/DRUIDA PROJECT/POC/dbGeneration_v0/Models/testing-multioutput/meta-atom_box_01_datageneration.aedt"
The new directory is created!


## Setting a dataframe to build our DB


In [23]:
now=datetime.now()
dbName=modelName+' batch'+'01'+" "+str(now)[:10]+'.csv'

dbManager=MG.DBManager(ansysPath=ansysPath,
                       modelName=modelName,
                       projectName=project_name,
                       designName=designName,
                       modelPath=modelPath, 
                       dbPath=dBPath,
                      dbName=dbName)


In [24]:


columnNames={ "sim_id":"object",
               "created_at":"datetime64[s]",
               "iteration":'int64',
               "modelName":'object',
                 "model":'object',
               "designName":'object',
               "type":'object',
               "layers":'object',
               "params":'object',
               "paramValues":'object',
               "fMin(GHz)":'float64',
               "fMax(GHz":'float64',
               "metric":'object', 
               "freq":'float64',
               "value":'float64'}


dbManager.load_df(columns=columnNames)


"""This object can be treated as a pandas dataframe"""
dbManager.df    
             

,sim_id,created_at,iteration,modelName,designName,type,layers,params,paramValues,fMin(GHz),fMax(GHz,metric,freq,value


## A sample of data structure to log in our DB


In [25]:
#create structs and data to store
layers= {
         "layer1":{
             "thickness":0.1,
             "material":"copper"
         },
         "layer2":{
             "thickness":0.8,
             "material":"dielectric1"
         } ,
         "layer2":{
             "thickness":0.1,
             "material":"copper"
         }
    }

params=['radius']
values=[0.3]


"""For trial. The real case needs to loop over the entire data from HFSS to load ech frequency point"""

data_to_store= {"sim_id":"simid123456",
            "created_at":"'1970-01-01T00:00:00Z'",
                "iteration":'0',
             "modelName":"meta-atom_box_01_datageneration",
            "model":"box",
             "designName":'HFSSDesign1',
             "type":'Reflective',
             "layers":layers,
             "params":params,
             "paramValues":values,
             "fMin(GHz)":5,
             "fMax(GHz)":10,
             "metric":"ReflectanceTE", 
             "freq":8.15,
            "value":0.556}




In [26]:
dbManager.insert_row(data_to_store)

# 2. A full batch of simulation with data storage

In [3]:


#import sys  
#sys.path.insert(0, './src/')

import os
import uuid 
from datetime import datetime


from __future__ import print_function
from druidaHFSS import Manager as MG
from druidaHFSS.modules import tools

from glob import glob
import matplotlib.pyplot as plt
import re


ansysPath="C:\\Program Files\\AnsysEM\\AnsysEM21.2\\Win64\\ansysedt.exe"
modelName="cross_01_freq_reflect"
scriptPath=r"C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\Models\\testing-multioutput\\scripts\\"


r"C:/Users/jorge/Documents/Projects Jorge C/DRUIDA PROJECT/POC/dbGeneration_v0/Models/testing-multioutput/"
modelPath=r"C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\Models\\testing-multioutput\\"
exportPath= "C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\Exports"
dBPath= "C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\DBfiles\\"
imagesPath="C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\Images"

#simulating box shaped meta atom
project_name="meta-atom_cross_01_datageneration"
designName="HFSSDesign1"
modeltype="Reflective"

#set simulation variables
#This is run to create iterative simulations
Builder=MG.Builder(ansysPath=ansysPath,
                   modelName=modelName,
                   projectName=project_name, 
                   designName=designName,
                   modelPath=modelPath,
                   scriptPath=scriptPath,
                   exportPath=exportPath,
                   imagesPath=imagesPath)



The new Exports directory is created!
The new Images directory is created!


In [4]:
"""If creating a model is required then:"""
Builder.create()



In [5]:
#Generate batch of vectors to simulate
"""This function must be custom made"""
import random
#[width, length, cond z, subs z, subs LxL]
def parameters_Generator(constraints, batch_size, vector_size):
    x = [[0 for i in range(vector_size)] for j in range(batch_size)]

    for vector in x:
        for i, (k, v) in enumerate(constraints.items()):
            if k!='zSustrato':
                vector[i]=random.uniform(constraints[k]["max"], constraints[k]["min"])
            else:
                 vector[i] = random.choice([1.575, 0.787, 0.508, 0.252 ])
             
    return x
    


In [9]:
#Simulation setting
batch=0
batch_size=1000
batches=1
iteration=0
simfile_path= "C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\"
simfileName= "intermediateFileCross.py"

#set reports
reports={
    "ReflectanceTE":"(mag(S(FloquetPort1:1,FloquetPort1:1)))^2",
    "ReflectanceTM":"(mag(S(FloquetPort1:2,FloquetPort1:2)))^2",
    "TransmittanceTE":"(mag(S(FloquetPort2:1,FloquetPort2:1)))^2",
    "TransmittanceTM":"(mag(S(FloquetPort2:2,FloquetPort2:2)))^2",
    "AbsorbanceTE":"1-((mag(S(FloquetPort1:1,FloquetPort1:1)))^2 +(mag(S(FloquetPort2:1,FloquetPort1:1)))^2)",
    "AbsorbanceTM":"1-((mag(S(FloquetPort1:2,FloquetPort1:2)))^2 +(mag(S(FloquetPort2:2,FloquetPort1:2)))^2)"
    }

#Rogers standars
#metalization 0.07mm, 0.035mm
#dielectric 1.575mm, 0.787 mm, 0.508mm, 0.252mm


constraints = {"arm_w":{"max":3.0, "min":0.1,"nominal":1},
            "arm_l":{"max":4.5, "min":2.0,"nominal":1},
            "zSize":{"max":0.035, "min":0.035,"nominal":0.5},
            "zSustrato":{"max":1.575, "min":0.252,"nominal":1},
            "Sustrato":{"max":5.3, "min":4.9,"nominal":5}}

"""name in the hfss design"""
variable_name='parameters'

parameters = parameters_Generator(constraints, batch_size=batch_size, vector_size=len(list(constraints.keys()) ))
parameters_names=list(constraints.keys())

#Data setting

now=datetime.now()
dbName=modelName+' batch'+str(batch)+" "+str(now)[:10]+'.csv'

dbManager=MG.DBManager(ansysPath=ansysPath,
                       modelName=modelName,
                       projectName=project_name,
                       designName=designName,
                       modelPath=modelPath, 
                       dbPath=dBPath,
                      dbName=dbName)





In [10]:


columnNames={ "sim_id":"object",
               "created_at":"datetime64[s]",
               "iteration":'int64',
               "projectName":'object',
                 "modelName":'object',
               "designName":'object',
               "type":'object',
               "layers":'object',
               "params":'object',
               "paramValues":'object',
             "units":'object',
               "fMin(GHz)":'float64',
               "fMax(GHz":'float64',
               "metric":'object', 
               "freq":'float64',
               "value":'float64'}


dbManager.load_df(columns=columnNames)


"""This object can be treated as a pandas dataframe"""
dbManager.df    
             

,sim_id,created_at,iteration,projectName,modelName,designName,type,layers,params,paramValues,units,fMin(GHz),fMax(GHz,metric,freq,value


In [ ]:
simulation_id=str(uuid.uuid1())

for parameter in parameters:
    
    #Describe the layers used in your model
    #create structs and data to store
    layers= {
             "conductor":{
                 "thickness":parameter[2],
                 "material":"copper"
             },
             "conductor2":{
                 "thickness":0,
                 "material":"pec"
             } ,
             "substrate":{
                 "thickness":parameter[3],
                 "material":"Rogers RT/duroid 5880 (tm)"
             }
        }
    
    #parameters, batch, iteration, filePath, **kwargs
    #Kwargs
    #AnsoftPath=kwargs['ansoft_path']
    #modelPath=kwargs['model_path'] 
    #outputPath=kwargs['output_path']
    #projectName=kwargs['project_name']
    #simulationID=kwargs['simulation_id']
    #variableName=kwargs['variable_name']
    #value=kwargs['value']
    #units=kwargs['units']
    #design=kwargs['design_name']

    #Create the intermediate simulation fiel
    
    

    #set variables to modify
    value=str(parameter)

    units="mm"
    
    kwargs={
        "reports":reports,
        "simulation_id":simulation_id,
       "variable_name":variable_name,
        "value" : value,
        "units" : units,
        "sim_file_name" : simfileName
       }
    
    Builder.sim_file('', batch, iteration, simfile_path, **kwargs)
    
    Builder.simulate(simfile_path+simfileName)
    

    """For trial. The real case needs to loop over the entire data from HFSS to load ech frequency point"""

    data_to_store= {"sim_id":str(simulation_id),
                "created_at":str(now)[:10],
                    "iteration":str(iteration),
                 "projectName":project_name,
                "modelName":modelName,
                 "designName":designName,
                 "type":modeltype,
                 "layers":layers,
                 "params":str(parameters_names),
                 "paramValues":str(value),
                    "units":str(units),
                 "fMin(GHz)":30,
                 "fMax(GHz)":90,
                 "metric":str(reports), 
                 "freq":'',
                "value":''}
    
    dbManager.insert_row(data_to_store)
    
    iteration=iteration+1


"C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\Models\\testing-multioutput\\meta-atom_cross_01_datageneration.aedt"
The new directory is created!
The new directory is created!
"C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\Models\\testing-multioutput\\meta-atom_cross_01_datageneration.aedt"
"C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\Models\\testing-multioutput\\meta-atom_cross_01_datageneration.aedt"
"C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\Models\\testing-multioutput\\meta-atom_cross_01_datageneration.aedt"
"C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\Models\\testing-multioutput\\meta-atom_cross_01_datageneration.aedt"
"C:\\Users\\jorge\\Documents\\Projects Jorge C\\DRUIDA PROJECT\\POC\\dbGeneration_v0\\Models\\testing-multioutput\\meta-atom_cross_01_datageneration.aedt"
"C:\\Users

## Post processing images


In [13]:

folders=glob(imagesPath+"/*/", recursive = True)
files=[]

for folder in folders:
    if folder != imagesPath+"\\"+ "processed\\":
        files=(files+glob(folder+"/*"))



In [14]:

for file in files:
    fileName_absolute = os.path.basename(file) 
    path=os.path.dirname(file)

    #ROI is 
    image_rgb=tools.cropImage( file,image_path=path,
                              image_name=fileName_absolute,
                              output_path=imagesPath, 
                             resize_dim=(512,512))
        


The new directory is created!
